In [12]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr_a
import tools.data_reader_case as dr_c
import tools.data_reader_wesad as dr_w
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

In [13]:
importlib.reload(dt)
importlib.reload(train)
importlib.reload(preprocessing)
importlib.reload(dr_a)
importlib.reload(dr_c)
importlib.reload(dr_w)

anxiety_label_type = None
case_label_type = dr_c.SelfReports.AROUSAL
wesad_label_type = "stai"

threshold = "fixed"
test_size = 1.0
percent_of_target_dataset = 0.0

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE
# ]
] + train.Metrics.STATISTICAL

model_phases_apd = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]
model_phases_wesad = dr_w.Phases.PHASE_ORDER

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr_a.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

temp_a, _ = train.Train_CASE.get_case_data(metrics, verbose=False, label_type=case_label_type, threshold=threshold, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_case_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_case_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

temp_a, _ = train.Train_WESAD.get_wesad_data([train.Metrics.BPM], phases=dr_w.Phases.PHASE_ORDER, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_wesad_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_wesad_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True, standardize=False)
x_c, y_c = train.Train_CASE.get_case_data(metrics, verbose=False, label_type=case_label_type, threshold=threshold, normalize=True, standardize=False)
x_w, y_w = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
# x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

if anxiety_label_type is not None:
    x_a = x_a.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

x_a = x_a.drop(["phaseId"], axis=1)
x_c = x_c.drop(["phaseId"], axis=1)
x_w = x_w.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_c).any(axis=1).to_numpy().nonzero()[0]
x_c = x_c.drop(labels=inds, axis=0).reset_index(drop=True)
y_c = y_c.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_w).any(axis=1).to_numpy().nonzero()[0]
x_w = x_w.drop(labels=inds, axis=0).reset_index(drop=True)
y_w = y_w.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_c["subject"] = x_c["subject"] + 500
y_c["subject"] = y_c["subject"] + 500
x_w["subject"] = x_w["subject"] + 1000
y_w["subject"] = y_w["subject"] + 1000

In [14]:
importlib.reload(dt)
importlib.reload(train)
importlib.reload(preprocessing)
importlib.reload(dr_a)
importlib.reload(dr_c)
importlib.reload(dr_w)

import statsmodels.api as sm
from statsmodels.regression.mixed_linear_model import MixedLM


endog = y_a.loc[:, "label"]
exog = x_a.loc[:, x_a.columns != "subject"]
groups = x_a.loc[:, "subject"]
model = MixedLM(endog, exog, groups)
result = model.fit()

print("APD " + "-"*49 + "\n")
print(result.summary())
print(result.pvalues)
print("\n")

######################################################################################

endog = y_c.loc[:, "label"]
exog = x_c.loc[:, x_c.columns != "subject"]
groups = x_c.loc[:, "subject"]
model = MixedLM(endog, exog, groups)
result = model.fit()

print("CASE " + "-"*49 + "\n")
print(result.summary())
print(result.pvalues)
print("\n")

######################################################################################

endog = y_w.loc[:, "label"]
exog = x_w.loc[:, x_w.columns != "subject"]
groups = x_w.loc[:, "subject"]
model = MixedLM(endog, exog, groups)
result = model.fit()

print("WESAD " + "-"*49 + "\n")
print(result.summary())
print(result.pvalues)
print("\n")

APD -------------------------------------------------

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: label    
No. Observations: 1034    Method:             REML     
No. Groups:       52      Scale:              0.1613   
Min. group size:  18      Log-Likelihood:     -587.9649
Max. group size:  29      Converged:          Yes      
Mean group size:  19.9                                 
-------------------------------------------------------
            Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
-------------------------------------------------------
bpm          1.053    0.138  7.637 0.000   0.783  1.323
rmssd       -0.721    0.389 -1.856 0.063  -1.483  0.040
hf_rr        0.253    0.489  0.519 0.604  -0.704  1.211
lf_rr       -0.301    0.463 -0.649 0.516  -1.209  0.607
sdnn         0.636    0.418  1.522 0.128  -0.183  1.455
mean_SCL     2.632    7.035  0.374 0.708 -11.156 16.421
SCR_rate    -0.469    0.101 -4.622 0.000  -0.668 -0.270
ec

In [15]:
importlib.reload(dt)
importlib.reload(train)
importlib.reload(preprocessing)
importlib.reload(dr_a)
importlib.reload(dr_c)
importlib.reload(dr_w)

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression


endog = y_a.loc[:, "label"]
exog = x_a.loc[:, x_a.columns != "subject"]
groups = x_a.loc[:, "subject"]
model = LinearRegression()
result = model.fit(exog, endog)

print("APD " + "-"*49 + "\n")
print(result.feature_names_in_)
print(result.coef_)
print(model.score(exog, endog))
print("\n")

######################################################################################

endog = y_c.loc[:, "label"]
exog = x_c.loc[:, x_c.columns != "subject"]
groups = x_c.loc[:, "subject"]
model = LinearRegression()
result = model.fit(exog, endog)

print("CASE " + "-"*49 + "\n")
print(result.feature_names_in_)
print(result.coef_)
print(model.score(exog, endog))
print("\n")

######################################################################################

endog = y_w.loc[:, "label"]
exog = x_w.loc[:, x_w.columns != "subject"]
groups = x_w.loc[:, "subject"]
model = LinearRegression()
result = model.fit(exog, endog)

print("WESAD " + "-"*49 + "\n")
print(result.feature_names_in_)
print(result.coef_)
print(model.score(exog, endog))
print("\n")

APD -------------------------------------------------

['bpm' 'rmssd' 'hf_rr' 'lf_rr' 'sdnn' 'mean_SCL' 'SCR_rate' 'ecg_mean'
 'ecg_median' 'ecg_std' 'ecg_var' 'eda_mean' 'eda_median' 'eda_std'
 'eda_var' 'lf_hf_ratio']
[  0.5071587   -0.90357718   0.99951951  -0.9987853    0.93645587
   2.99931711  -0.20345403 -14.83322406  15.15027964   0.14211609
  -0.22758108  -2.94144333  -0.23560223   0.48371969  -0.42993913
   0.10755606]
0.1016106001530287


CASE -------------------------------------------------

['bpm' 'rmssd' 'hf_rr' 'lf_rr' 'sdnn' 'mean_SCL' 'SCR_rate' 'ecg_mean'
 'ecg_median' 'ecg_std' 'ecg_var' 'eda_mean' 'eda_median' 'eda_std'
 'eda_var' 'lf_hf_ratio']
[ 0.5990301  -0.08465856 -0.12425708  0.02511222  0.37275751 -0.02728299
  0.25399485 -2.1524145   1.9650399  -1.26760178  0.36408978  3.63147125
 -3.72655508  0.40970443  0.24198779  0.03699836]
0.3100271009403551


WESAD -------------------------------------------------

['bpm' 'rmssd' 'hf_rr' 'lf_rr' 'sdnn' 'mean_SCL' 'S